# Algo TP3 

# Ex 1. Chaine de 0 et 1 qui resoud un UBQP
---

### Reading Bit file ex `graphe_.txt`

In [50]:
# Read file
def read_file_ubgp(file_path):
    f = open(file_path, "r")
    
    content = f.read()
    content = content.replace("\n", "")
    content_list = content.split(" ")
    content_list = list(map(int, content_list))
    
    n = content_list[0]
    p = content_list[1]
    Q = np.array(content_list[2:])
    Q.shape = ((n,n))
    
    f.close()
    return(n,p,Q)

In [51]:
import numpy as np

def create_rand_bit_vector(n):
    return [np.random.choice([0,1]) for i in range(n)]

### Neighbours

In [52]:
# Unconstrained Binary Quadratic Problem
def UBGP(matrix,X):        
    #result = np.apply_along_axis((lambda row: row*X), axis=1, arr=matrix).sum(axis=1)
    res = 0
    for i in range(len(matrix)):
        for j in range(len(matrix[0])):
            res += matrix[i,j]*X[i]*X[j]
        
    return res

In [53]:
def voisins(s):
    voisins = []
    temp = []
    for i in range(len(s)):
        temp = s.copy()
        temp[i] = int(not temp[i])
        voisins.append(temp) 
    return voisins

In [54]:
def meilleur_voisin(X,p,Q):
    solutions  = []
    neighbours = []
    neighbours = voisins(X)
    s = neighbours[0]
    for i in range(1,len(neighbours)):
        #diff = (np.array(neighbour) == np.array(X))
        #if(sum(diff) >= len(X)-1 and sum(diff)>=p):
        if(sum(neighbours[i])>=p and UBGP(Q,neighbours[i])<=UBGP(Q,s)):
            s = neighbours[i]
    #randomRow = np.random.randint(len(solutions), size=1)
    #return (solutions[randomRow[0]])
    return s

In [55]:
def meilleur(s1, s2, Q): #MIN OPT?
    return(UBGP(Q,s1)<UBGP(Q,s2))

In [84]:
def voisins_non_Tabou(s,tabou):
    voisins = []
    temp = []
    for i in range(len(s)):
        temp = s.copy()
        temp[i] = int(not temp[i])
        if(temp not in tabou):
            voisins.append(temp) 
    return voisins

In [109]:
def meilleur_voisin_non_tabou(X,Q,p,t):
    solutions  = []
    neighbours = []
    neighbours = voisins_non_Tabou(X,t)
    s = neighbours[0]
    for i in range(1,len(neighbours)):
        if(sum(neighbours[i])>=p and UBGP(Q,neighbours[i])<=UBGP(Q,s)):
            s = neighbours[i]
    return s

### Algorithms 

In [110]:
def steepest_hill_climbing(s,Q,p,MAX_depl):
    nb_depl = 0
    stop = False
    print("Solution inital : ", s)
    while(True):
        new_s = meilleur_voisin(s,p,Q)
        if(meilleur(new_s,s,Q)):
            s = new_s
        else:
            stop = True # optimum local
        nb_depl += nb_depl
        if(stop or nb_depl == MAX_depl):
            break;
        print("Solution atteinte ",s,", avec nb deplacement: ", MAX_depl)
    return s

In [111]:
from queue import Queue

def tabou(s,Q,p,MAX_depl,k):
    #tabou = Queue(maxsize = k)
    tabou = []
    nb_depl = 0
    msol = s
    stop = False
    while(True):
        if(voisins_non_Tabou(s,tabou)):
            new_s = meilleur_voisin_non_tabou(s,Q,p,tabou) #plus de voisin non tabou
        else:
            stop = True
        #tabou.put(s)
        tabou.append(s)
        tabou = tabou[:k] #max size 
        if(meilleur(new_s, msol, Q)):
            msol = new_s #stockage meilleure solution courante
            s = new_s
            nb_depl += nb_depl
        if(nb_depl == MAX_depl or stop):
            break
    return msol        

### Main Bit

In [ ]:
#---- read from file vector size N, verification P and matrix Q 
n,p,Q = read_file_ubgp("data/partition6.txt")

MAX_essais = 3
MAX_depl = 3

solutions = []
solutions_ubgp = []
s = [1, 0, 0, 0, 1, 0]


#---- for each essais make max depl to find MIN opt in Steepest Hill
for i in range(MAX_essais):
    solutions.append(steepest_hill_climbing(s,Q,p,MAX_depl))
    s = create_rand_bit_vector(n)

#---- print all essais solutions
for sol in solutions:
    ubgp = UBGP(Q,sol)
    solutions_ubgp.append(ubgp)
    print(sol,": ",ubgp)

#---- from all essais solutions which one is best
print("Optimal : ",solutions[solutions_ubgp.index(min(solutions_ubgp))])

#---- from all essais solutions which one is best
print(tabou(s,Q,p,MAX_depl,1))

# Ex 2. Traveling Salesman
---

### Reading TSP file ex. `tsp_.txt`

In [34]:
# Read file
def read_file_tsp(file_path):
    f = open (file_path , 'r')
    l = [line.split() for line in f]
    #l.astype(int)
    #l = list(map(int, l))
    n = l[0][0]
    villes = l[1:]
    f.close()
    return(n,villes)

In [35]:
import numpy as np

def chemin_random(villes):
    chemin = []
    copy_villes = villes.copy()
    for i in range(len(copy_villes)):
        #chemin = np.random.shuffle(villes)
        randomRow = np.random.randint(len(copy_villes), size=1)
        chemin.append(copy_villes[randomRow[0]])
        del copy_villes[randomRow[0]]
    return chemin

###  Calculating distance 

In [36]:
import math

def eucledian(vector1, vector2):
    return math.sqrt((int(vector1[1]) - int(vector2[1]))**2 + (int(vector1[2]) - int(vector2[2]))**2)

In [37]:
def distance(chemin):
    distance = 0;
    for i in range(0,len(chemin)-1):
        distance += eucledian(chemin[i],chemin[i+1])
    return distance

In [38]:
import itertools

def meilleur_voisin_tsp(X,villes):
    msol = villes[0]
    #permutations = list(itertools.permutations(villes, r=2))
    for i in range(1,len(villes)):
        if(eucledian(X, villes[i]) < eucledian(X, msol)):
            msol = villes[i]
    return (msol,eucledian(X, msol))

In [39]:
def meilleur_tsp(ville1, ville2):
    return distance(ville1) <  distance(vllle2)

### Steepest Hill Climbing for TSP

In [40]:
def steepest_hill_climbing_tsp(s,villes,MAX_depl):
    nb_depl = 0
    stop = False
    print("Solution initial ",s)
    while(True):
        new_s = meilleur_voisin_tsp(s,villes)
        if(meilleur(new_s,s)):
            s = new_s
        else:
            stop = True # optimum local
        nb_depl += nb_depl
        if(not stop and nb_depl != MAX_depl):
            break;
    print("Solution atteinte ",s, ", avec nb deplacement: ", nb_depl)
    return s

### Tabou TSP

In [41]:
import queue

def tabou_tsp(s, k):
    tabou = queue.Queue()
    nb_depl = 0
    msol = s
    stop = false
    while(True):
        if(not voisins_non_Tabou(s)):
            new_s = meilleur_voisin_non_Tabou(s) #plus de voisin non tabou
        else:
            stop = True
        tabou.put(s)
        if(meilleur(new_s, msol)):
            msol = new_s #stockage meilleure solution courante
            s = new_s
            nb_depl += nb_depl
        if(nb_depl != MAX_depl and not stop):
            break
    return msol

### Main TSP

In [42]:
n, villes = read_file_tsp("data/tsp5.txt")
print(n,villes)
chemin = chemin_random(villes)
print(chemin)
d = distance(chemin)
print(d)
print(meilleur_voisin_tsp(villes[0],villes[1:]))

5 [['1', '40', '50'], ['2', '25', '85'], ['3', '22', '75'], ['4', '22', '85'], ['5', '20', '80']]
[['3', '22', '75'], ['1', '40', '50'], ['4', '22', '85'], ['2', '25', '85'], ['5', '20', '80']]
80.23424872167305
(['3', '22', '75'], 30.805843601498726)
